## 유동인구 데이터 수집후 컨켓

In [ ]:
import os
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for root, dirs, files in os.walk('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구 데이터/유동인구 컨켓 폴더'):
    print('### root:', root)
    print('--- dirs: ', dirs)
    print(files)

In [ ]:
%%time

def living_concat(root_path):
    #root_path: 상위폴더 경로
    root_path = root_path
    folder_list = [name for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, name))]
    
    #dataframe 생성
    df_total = pd.DataFrame()
    
    for folder in folder_list:
        file_list = os.listdir(os.path.join(root_path, folder))
        file_list_csv = [file for file in file_list if file.endswith('.csv')] # csv만 추출
        
        start_time=time.time()
        # 파일읽기
        print("### Selected Directory:", folder)
        
        for file in file_list_csv:
            try:
                df = pd.read_csv(os.path.join(root_path, folder, file),
                                index_col=False, header=0,
                                names=['기준일ID', '시간대구분', '행정동코드', '집계구코드', '총생활인구수'], 
                                encoding='UTF-8', usecols=[0,1,2,3,4])
            except:
                df = pd.read_csv(os.path.join(root_path, folder, file),
                                index_col=False, header=0,
                                names=['기준일ID', '시간대구분', '행정동코드', '집계구코드', '총생활인구수'], 
                                encoding='cp949', usecols=[0,1,2,3,4])
            # 데이터 확인 코드
            print('파일명:' + file) 
            print("집계구 개수:",df['집계구코드'].nunique())
            print("시간대구분 개수:", df['시간대구분'].nunique())
            print("기준일:", df['기준일ID'].nunique())
            
            df_total = pd.concat([df_total, df], axis = 0, ignore_index=False)
            

        
        #데이터 확인 코드
        print("기준일 토탈:", df_total['기준일ID'].nunique())
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print("경과 시간: ", elapsed_time, "초\n")       
    
    return df_total

In [ ]:
%%time
df_total = living_concat('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구 데이터/202304_20230531')

In [ ]:
%%time
# groupby로 집계구코드, 날짜를 기준으로 총생활인구수 합치기
df_day = df_total.groupby(['집계구코드', '기준일ID'], as_index=False)[['총생활인구수']].sum()

In [ ]:
print(df_day['기준일ID'].nunique())
df_day.head()

In [ ]:
#파일 저장코드
df_day.to_csv('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구 데이터/livning_pop_202304_05.csv', index=False, encoding='euc-kr')

# 유동인구 데이터와 스타벅스 데이터 조인

In [ ]:
liv = pd.read_csv('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구_스타벅스/living_pop.csv',
                 encoding = 'euc-kr')
liv.head()

In [ ]:
liv.isnull().sum()

In [ ]:
# 기준일ID datetype으로 전환
def transform_datetype(df):
    df['기준일ID'] = df['기준일ID'].astype('str')
    df['기준일ID'] = pd.to_datetime(df['기준일ID'])
    return df

In [ ]:
liv = transform_datetype(liv)

In [ ]:
print(liv.info())
liv.head()

## 스타벅스 데이터 불러오기

In [ ]:
star_area = pd.read_csv('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구_스타벅스/스타벅스별 집계구 정보.csv',
                       encoding='cp949')
star_area.head()

In [ ]:
star_area.info()

In [ ]:
# 인허가일자 datetype으로 전환
star_area['인허가일자'] = star_area['인허가일자'].astype('str')
star_area['인허가일자'] = pd.to_datetime(star_area['인허가일자'])
print(star_area.info())
print(star_area)

## 유동인구 3개월 전후 평균 컬럼 만들기

In [ ]:
from dateutil.relativedelta import relativedelta
def pop_mean(area, data, day):
    # area : 집계구코드
    # data : 총생활인구수 파일
    # day : 개점일자
    df = data[data['집계구코드'] == area]
    
    #day datetime으로
    #day = pd.to_datetime(str(day))
    
    #3개월이후
    mask_plus3 = (df['기준일ID'] >= day)&(df['기준일ID'] <= day + relativedelta(months=3))
    #3개월이전
    mask_minus3 = (df['기준일ID'] >= day - relativedelta(months=3))&(df['기준일ID'] <= day)

    # 3개월 이후까지의 총생활인구수 평균
    h = df[mask_plus3]['총생활인구수'].mean()
    
    # 3개월 이후까지의 총생활인구수 평균
    m = df[mask_minus3]['총생활인구수'].mean()
    
    k = [h, m]
    return k

    

In [ ]:
#####스타벅스, 유동인구 조인후 3개월 3개월전 평균 구하기
pop_after = []
pop_before = []

for day, area in zip(star_area['인허가일자'], star_area['TOT_REG_CD']):
    k = pop_mean(area, liv, day)
    pop_after.append(k[0])
    pop_before.append(k[1])
    
    

In [ ]:
star_area = star_area.assign(pop_plus = pop_after,pop_minus = pop_before)

In [ ]:
data_s = star_area[['인허가일자','사업장명', 'pop_plus', 'pop_minus']]

In [ ]:
#파일저장
data_s.to_csv('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구_스타벅스/living_fin.csv', index=False, encoding='euc-kr')

## 유동인구_스타벅스 최종파일

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 파일 불러오기

In [ ]:
base_df = pd.read_csv('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/임시데이터셋.csv',
                 encoding='euc-kr')
base_df.head()

In [ ]:
base_df.describe()

## 공시지가 유동인구 차이값 행 생성

In [ ]:
# 3개월후까지 합한값과 값과 3개월전까지 합한값의 유동인구 차이를 퍼스센트화
base_df['pop_per'] = ((base_df['pop_plus'] - base_df['pop_minus']) / base_df['pop_plus']) *100

In [ ]:
# 공시지가 전후 차이
base_df['price_dff'] = ((base_df['후_공시지가']-base_df['전_공시지가'])

In [ ]:
# 서브플롯 생성
fig, axs = plt.subplots(2, 1, figsize=(8,6))  # 2개의 그래프를 세로로 배치

# 첫 번째 그래프
sns.boxplot(x=base_df['pop_per'], ax=axs[0])
axs[0].set_title('pop')

# 두 번째 그래프
sns.boxplot(x=base_df['price_dff'], ax=axs[1])
axs[1].set_title('price')

# 그래프 간 간격 조정
plt.tight_layout()

# 그래프 표시
plt.show()

In [ ]:
base_df.to_csv('/Users/kwakdong-u/Library/CloudStorage/OneDrive-UOS/스타벅스 유동인구 팀플/유동인구_스타벅스/전처리_데이터_peradd.csv', index=False, encoding='euc-kr')